In [2]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from glob import glob
import os

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import transforms as T
from torchvision.transforms.functional import InterpolationMode
from tensorflow.keras.utils import to_categorical

#---------------------------------------
import custom_dataset
import utils_rs

#----------------------------------------------
import swin
import upper_net_mmseg
import models


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# model swin
model_swin = swin.swin()

# model upernet
model_upernet = upper_net_mmseg.UPerHead(
    in_channels = model_swin.out_channels[1:],
    channels = model_swin.out_channels[2],
    in_index= (0,1,2,3),
    dropout_ratio=0.1,
    norm_cfg= dict(type='SyncBN', requires_grad=True)
)


# model samrs 
swin_samrs = models.SamRS(model1=model_swin, model2=model_upernet)

In [4]:

ISAID_CLASSES = ('background', 'ship', 'store_tank', 'baseball_diamond',
               'tennis_court', 'basketball_court', 'Ground_Track_Field',
               'Bridge', 'Large_Vehicle', 'Small_Vehicle', 'Helicopter',
               'Swimming_pool', 'Roundabout', 'Soccer_ball_field', 'plane',
               'Harbor')

ISAID_PALETTE = {
    0: (0, 0, 0), 1: (0, 0, 63), 2: (0, 63, 63), 3: (0, 63, 0), 4: (0, 63, 127),
    5: (0, 63, 191), 6: (0, 63, 255), 7: (0, 127, 63), 8: (0, 127, 127),
    9: (0, 0, 127), 10: (0, 0, 191), 11: (0, 0, 255), 12: (0, 191, 127),
    13: (0, 127, 191), 14: (0, 127, 255), 15: (0, 100, 155)}


In [5]:
img_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs"
mask_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all"

assert len(os.listdir(img_path)) == len(os.listdir(mask_path))

segdataset = custom_dataset.SegDataset(img_path, mask_path, 512)

['/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_0.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_1.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_0_2.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/04.1024_imgs/P0000_1_0.png']
['/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/0.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/1.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/2.png', '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/08.1024_masks_categorized_imgs_all/3.png']


In [6]:
dummy_ = torch.randn((1,3,512,512))

In [7]:
dummy_.shape

torch.Size([1, 3, 512, 512])

In [8]:
#-----------------------------------------

import wandb
import logging
from tqdm import tqdm
from lightning.fabric import Fabric
import lightning as L 

In [9]:
# distribute gpus
device_list = [0,1,2,3]
fabric = L.Fabric(accelerator="cuda", devices=device_list, strategy='ddp')
fabric.launch()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/lightning/fabric/fabric.py:12 │
│ 9 in __init__                                                                                    │
│                                                                                                  │
│    126 │   │   callbacks: Optional[Union[List[Any], Any]] = None,                                │
│    127 │   │   loggers: Optional[Union[Logger, List[Logger]]] = None,                            │
│    128 │   ) -> None:                                                                            │
│ ❱  129 │   │   self._connector = _Connector(                                                     │
│    130 │   │   │   accelerator=accelerator,                                                      │
│    131 │   │   │   strategy=strategy,                                                            │
│    132 │   │   │   devices=devices,                                                              │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/lightning/fabric/connector.py │
│ :160 in __init__                                                                                 │
│                                                                                                  │
│   157 │   │   self.precision = self._check_and_init_precision()                                  │
│   158 │   │                                                                                      │
│   159 │   │   # 6. Instantiate Strategy - Part 2                                                 │
│ ❱ 160 │   │   self._lazy_init_strategy()                                                         │
│   161 │                                                                                          │
│   162 │   def _check_config_and_set_final_flags(                                                 │
│   163 │   │   self,                                                                              │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/lightning/fabric/connector.py │
│ :515 in _lazy_init_strategy                                                                      │
│                                                                                                  │
│   512 │   │   self.strategy._configure_launcher()                                                │
│   513 │   │                                                                                      │
│   514 │   │   if _IS_INTERACTIVE and self.strategy.launcher and not self.strategy.launcher.is_   │
│ ❱ 515 │   │   │   raise RuntimeError(                                                            │
│   516 │   │   │   │   f"`Fabric(strategy={self._strategy_flag!r})` is not compatible with an i   │
│   517 │   │   │   │   " environment. Run your code as a script, or choose one of the compatibl   │
│   518 │   │   │   │   f" `Fabric(strategy='dp'|'ddp_notebook')`."                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: `Fabric(strategy='ddp')` is not compatible with an interactive environment. Run your code as a 
script, or choose one of the compatible strategies: `Fabric(strategy='dp'|'ddp_notebook')`. In case you are 
spawning processes yourself, make sure to include the Fabric creation inside the worker function.

In [10]:
# log 
logging.basicConfig(filename='./1.log/model_v1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [12]:
# opt 
optimizer = torch.optim.AdamW(swin_samrs.parameters(), lr=1e-5)

# model 
swin_samrs = swin_samrs.train()


# dataset 
batch_size= 4 
segdataset = custom_dataset.SegDataset(img_path, mask_path, 512)
dataloader = torch.utils.data.DataLoader(segdataset, batch_size=batch_size, shuffle=True)
dataloader = fabric.setup_dataloaders(dataloader)

# Define loss
criterion = nn.CrossEntropyLoss()


# run 
epochs = 999
for epoch in range(epochs):

    iteration = 0
    running_loss = 0 
    
    for i, data in enumerate(tqdm(dataloader, desc=f"Epoch {epoch}")):

        img, mask = data

        # opt 
        optimizer.zero_grad()
        
        # run
        outputs = swin_samrs(img)
        
        # criterion
        
        loss = criterion(outputs, mask)
        #print(loss)
        
        #Loss.backward()
        fabric.backward(loss)
        optimizer.step()

        # stat
        running_loss += loss.item()

        # log
        logger.info(f"[{epoch}, {i}] loss: {loss:.8f}")

        
        log = {'loss': f'{loss / 10:.8f}' }
        #print(log)
        wandb.log(log)

        log_iter = 200
        if (i % log_iter) == 0:    # print every 2000 mini-batches
            print(f"epoch : {epoch} iter : {i} /  total_iter : {len(dataloader)} running_loss : {running_loss / log_iter}")
            
            running_loss = 0.0
         
            
        #-----
    #-- epoch
    save_path = f"./2.ckpts/swin_rs_{epoch + 1}.pt"
    torch.save(swin_samrs.state_dict(), save_path)




SamRS(
  (encoder): swin(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=True)
            